In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install openpyxl
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reddit_sample = pd.read_csv('/kaggle/input/donald-trump-comments-on-reddit/reddit_trump.csv', error_bad_lines=False)
twitter_sample = pd.read_csv("/kaggle/input/twitter/Sentiment.csv")
#data = pd.read_csv('file1.csv', error_bad_lines=False)
reddit_sample = pd.read_csv("/kaggle/input/redditsentiments/reddit_sentiment.csv") 
youtube_sample = pd.read_csv("/kaggle/input/youtubesentiments/youtube_sentiment.csv")
generic_sample = pd.read_csv("/kaggle/input/genericcomments/generic_sentiment.csv")
#reddit_comments = pd.read_csv("/kaggle/input/genericsentiment/generic_sentiment.csv")
reddit_comments = pd.read_excel('/kaggle/input/redditredo/redditcomments.xlsx', index_col=0)
youtube_comments = pd.read_csv("/kaggle/input/youtuberedo/data.csv")

In [ ]:
reddit_comments.head()

In [ ]:
reddit_sample = pd.read_csv("/kaggle/input/redditsentiments/reddit_sentiment.csv") 
reddit_sample.info()
youtube_sample = pd.read_csv("/kaggle/input/youtubesentiments/youtube_sentiment.csv")
youtube_sample.info()
twitter_sample.info()

In [ ]:
youtube_sample = youtube_sample.head(5000)
youtube_sample.info()
reddit_sample.info()
twitter_sample = twitter_sample.head(5000)
twitter_sample.info()
generic_sample.info()

In [ ]:
df = pd.DataFrame({
    'Titter': [76,78,80,79,80],
    'Reddit': [67,68,62,61,66],
    'Youtube': [90,91,88,88,88],
    'Generic': [80,82,61,60,62],
   }, index=['First ML', 'Second ML', 'MN NB', 'LR', 'SVM'], )
lines = df.plot.line(title="Testing #1 graphical representation", legend=False)
lines.set_xlabel("Algorithm")
lines.set_ylabel("Accuracy (%)")
#lines.title("Testing #1 graphical representation")
#df.suptitle("Testing #1 graphical representation")

Making the generic dataset 1.6k of each

In [ ]:

twitter_samplet = twitter_sample.head(2000)
dft = df.head(2000)
youtube_commentst = youtube_comments.head(2000)

list_of_t_c = twitter_samplet['correct_size'].values
list_of_t_s = twitter_samplet['sentiment'].values
list_of_r_c = dft['correct_size'].values
list_of_r_s = dft['comp_score'].values
list_of_y_c = youtube_commentst['correct_length'].values
list_of_y_s = youtube_commentst['comp_score'].values
list_of_g_c = []
list_of_g_s = []

for thing in list_of_t_c:
    list_of_g_c.append(thing)
for thing in list_of_r_c:
    list_of_g_c.append(thing)
for thing in list_of_y_c:
    list_of_g_c.append(thing)

for thing in list_of_t_s:
    list_of_g_s.append(thing)
for thing in list_of_r_s:
    list_of_g_s.append(thing)
for thing in list_of_y_s:
    list_of_g_s.append(thing)    

print(len(list_of_g_c))
print(len(list_of_g_s))

generic_sentimentredo = pd.DataFrame(data=list_of_g_c, columns = ['comments'])
generic_sentimentredo['sentiment'] = list_of_g_s
generic_sentimentredo['comments'] = list_of_g_c

generic_sentimentredo.to_csv('generic_sentimentredo.csv', index=False)

In [ ]:
generic_sentimentredo.info()

In [ ]:
list_of_g_c.append(list_of_t_c)
list_of_g_c.append(list_of_r_c)
list_of_g_c.append(list_of_y_c)

list_of_g_s.append(list_of_t_s)
list_of_g_s.append(list_of_r_s)
list_of_g_s.append(list_of_y_s)

Youtube processing

In [ ]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

youtube_comments['text'] = youtube_comments['text'].apply(lambda x:clean_text(x))

import emoji
import spacy
import stop_words
s = youtube_comments['text']
list_of_list = []
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:2] == "rt":
            my_list == my_list
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
            
    list_of_list.append(my_list)

#print(list_of_list.values)
youtube_comments['processing_text'] = list_of_list

In [ ]:
list_of_c = youtube_comments['processing_text'].values
list_of_r = []

for comment in list_of_c:
    cplit = comment.split()
    if len(cplit) < 3:
        list_of_r.append('NaN')
    elif len(cplit) > 27:
        list_of_r.append(cplit[:27])
    else:
        list_of_r.append(comment)
        
print(len(list_of_c))
print(len(list_of_r))

In [ ]:
youtube_comments['correct_length'] = list_of_r
youtube_comments = youtube_comments[youtube_comments['correct_length'].notna()]
youtube_comments = youtube_comments[youtube_comments.correct_length != "NaN"]

In [ ]:
youtube_sample = youtube_sample[youtube_sample.correct_length != "NaN"]

In [ ]:
youtube_comments.info()

In [ ]:
reddit_comments = pd.read_excel('/kaggle/input/redditredo/redditcomments.xlsx', index_col=0)
reddit_comments = reddit_comments[reddit_comments['comments'].notna()]
reddit_comments.head(10)

Reddit data processing

In [ ]:
list_of_com = reddit_comments['comments'].values
print(len(list_of_com))
complete_list = []
temp = []
#print(len(temp))

for com in list_of_com:  
    com = str(com)
    if com[0:12] == '[+][deleted]': 
        complete_list.append('NaN')
    elif com[0:9] == 'load more':
        complete_list.append('NaN')
    elif com[0:3] == '[–]':
        complete_list.append('NaN')
    elif com[0:3] == '[+]':
        complete_list.append('NaN')
    else:
        split_com = com.split()    
        if len(split_com) > 1:
            checker = split_com[0]
            #print(checker[0:9])
            if checker[0:9] == 'permalink':
                #print("Yes")
                if len(temp) == 0:
                    temp.append(com)
                else:
                    complete_list.append(temp)
                    temp = []
                    temp.append(com)
                    #print("added new")
            else:
                temp.append(" " + com)
        
        
print(len(complete_list))
#tempdf = 
#df = df[df.correct_size != "NaN"]

In [ ]:
comment_list = []

for com in complete_list:   
    com = str(com)
    if com[0:3] == 'NaN':
        comment_list.append('NaN')
    #print(com)
    elif com[2:11] == 'permalink':
        brack_split_com = com.split('children)')
        #print(len(brack_split_com))
        if len(brack_split_com) == 2:
                comment_list.append(brack_split_com[1])
        elif len(brack_split_com) == 1:
            #print(brack_split_com)
            single_child_split_com = com.split('child)')
            if len(single_child_split_com) == 2:
                comment_list.append(single_child_split_com[1])
 

print(len(comment_list))

In [ ]:
redditredo = pd.DataFrame(data=comment_list, columns = ['comments'])
redditredo = redditredo[redditredo.comments != "NaN"]
redditredo.info()

In [ ]:
redditredo.info()

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
redditredo['scores'] = redditredo['comments'].apply(lambda review: sid.polarity_scores(review))
redditredo['compound']  = redditredo['scores'].apply(lambda score_dict: score_dict['compound'])
redditredo['comp_score'] = redditredo['compound'].apply(lambda c: 'positive' if c >=0 else 'negative')

redditredo.head()

In [ ]:
youtube_comments['scores'] = youtube_comments['text'].apply(lambda review: sid.polarity_scores(review))
youtube_comments['compound']  = youtube_comments['scores'].apply(lambda score_dict: score_dict['compound'])
youtube_comments['comp_score'] = youtube_comments['compound'].apply(lambda c: 'positive' if c >=0 else 'negative')

youtube_comments.head()

In [ ]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

redditredo['comments'] = redditredo['comments'].apply(lambda x:clean_text(x))

import emoji
import spacy
import stop_words
s = redditredo['comments']
list_of_list = []
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:2] == "rt":
            my_list == my_list
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
            
    list_of_list.append(my_list)

#print(list_of_list.values)
redditredo['correct_length'] = list_of_list

In [ ]:
redditredo.head()

In [ ]:
list_of_c = redditredo['correct_length']
list_of_r = []

for comment in list_of_c:
    cplit = comment.split()
    if len(cplit) < 3:
        list_of_r.append('NaN')
    elif len(cplit) > 27:
        list_of_r.append(cplit[:27])
    else:
        list_of_r.append(comment)
        
print(len(list_of_c))
print(len(list_of_r))

In [ ]:
redditredo['correct_size'] = list_of_r

In [ ]:
redditredo = redditredo[redditredo.correct_size != "NaN"]
redditredo.info()
#twitter_sample = twitter_sample[twitter_sample.sentiment != "Neutral"]

In [ ]:
df.info()

Twitter data processing

In [ ]:
twitter_sample.drop(columns=['relevant_yn'], inplace=True)
twitter_sample.drop(columns=['relevant_yn_confidence'], inplace=True)
twitter_sample.drop(columns=['subject_matter'], inplace=True)
twitter_sample.drop(columns=['subject_matter_confidence'], inplace=True)
twitter_sample.drop(columns=['candidate_gold'], inplace=True)
twitter_sample.drop(columns=['relevant_yn_gold'], inplace=True)
twitter_sample.drop(columns=['retweet_count'], inplace=True)
twitter_sample.drop(columns=['sentiment_gold'], inplace=True)
twitter_sample.drop(columns=['subject_matter_gold'], inplace=True)
twitter_sample.drop(columns=['tweet_coord'], inplace=True)
twitter_sample.drop(columns=['tweet_created'], inplace=True)
twitter_sample.drop(columns=['tweet_id'], inplace=True)
twitter_sample.drop(columns=['user_timezone'], inplace=True)
twitter_sample.drop(columns=['name'], inplace=True)
twitter_sample.drop(columns=['tweet_location'], inplace=True)
#twitter_sample = twitter_sample.drop(twitter_sample[twitter_sample.candidate != 'Donald Trump'].index)
twitter_sample = twitter_sample[twitter_sample.sentiment != "Neutral"]
twitter_sample = twitter_sample.head(10000)
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

twitter_sample['text'] = twitter_sample['text'].apply(lambda x:clean_text(x))

import emoji
import spacy
import stop_words
s = twitter_sample['text'].values
list_of_list = []
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:2] == "rt":
            my_list == my_list
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
            
    list_of_list.append(my_list)

#print(list_of_list.values)
twitter_sample['processing_text'] = list_of_list

twitter_sample.drop(columns=['id'], inplace=True)
twitter_sample.drop(columns=['candidate'], inplace=True)
twitter_sample.drop(columns=['candidate_confidence'], inplace=True)
twitter_sample.drop(columns=['sentiment_confidence'], inplace=True)
twitter_sample.drop(columns=['text'], inplace=True)

list_of_c = twitter_sample['processing_text']
list_of_r = []

for comment in list_of_c:
    cplit = comment.split()
    if len(cplit) < 3:
        list_of_r.append('NaN')
    elif len(cplit) > 27:
        list_of_r.append(cplit[:27])
    else:
        list_of_r.append(comment)
        
print(len(list_of_c))
print(len(list_of_r))

twitter_sample['correct_size'] = list_of_r

twitter_sample = twitter_sample[twitter_sample.correct_size != "NaN"]
twitter_sample.info()

In [ ]:
#twitter_sample['candidate'].unique()
twitter_sample.head(6)

For trump training

In [ ]:
twitter_sample = twitter_sample.drop(twitter_sample[twitter_sample.candidate != 'Donald Trump'].index)

for training and assigning sentiment

In [ ]:
twitter_sample = twitter_sample.head(5000)

Tensorflow model #1

In [ ]:
youtube_sample.info()

In [ ]:
twitter_sample = twitter_sample.head(5000)
youtube_sample = youtube_sample.head(5000)

twitter_sample.info()
reddit_sample.info()
youtube_sample.info()
generic_sample.info()

In [ ]:
reddit_sample.head(50)

In [ ]:
sentiment = generic_sample['sentiment'].values
#print(sentiment)
neg = []
pos = []

for sent in sentiment:
    if sent == 'Positive':
        pos.append('pos')
    elif sent == 'Negative':
        neg.append('neg')
        
print(len(neg))
print(len(pos))

In [ ]:
twitter_sample.info()
df.info()
youtube_comments.info()

twitter_sample = twitter_sample.head(6000)
df = df.head(6000)

youtube_comments = youtube_comments.head(6000)
youtube_comments.info()
df.info()
twitter_sample.info()

In [ ]:
twitter_sample.head()

In [ ]:
redditredo.head()

In [ ]:
youtube_comments.head()

In [ ]:
generic_sentimentredo.info()
generic_sentimentredo['sentiment'] = generic_sentimentredo['sentiment'].replace(1, 'positive')
generic_sentimentredo['sentiment'] = generic_sentimentredo['sentiment'].replace(0, 'negative')
generic_sentimentredo['sentiment'] = generic_sentimentredo['sentiment'].replace('Positive', 'negative')
generic_sentimentredo['sentiment'] = generic_sentimentredo['sentiment'].replace('Negative', 'negative')

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(generic_sentimentredo['comments'].values)
X = tokenizer.texts_to_sequences(generic_sentimentredo['comments'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

modelg = Sequential()
modelg.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
modelg.add(SpatialDropout1D(0.4))
modelg.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
modelg.add(Dense(2,activation='sigmoid'))
modelg.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(modelg.summary())

Y = pd.get_dummies(generic_sentimentredo['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 32
modelg.fit(X_train, Y_train, epochs = 40, batch_size=batch_size, verbose = 2)

validation_size = 800

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = modelg.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(youtube_comments['correct_length'].values)
X = tokenizer.texts_to_sequences(youtube_comments['correct_length'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

modely = Sequential()
modely.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
modely.add(SpatialDropout1D(0.4))
modely.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
modely.add(Dense(2,activation='sigmoid'))
modely.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(modely.summary())

Y = pd.get_dummies(youtube_comments['comp_score']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 32
modely.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 2)

validation_size = 800

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = modely.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['correct_size'].values)
X = tokenizer.texts_to_sequences(df['correct_size'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

modelr = Sequential()
modelr.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
modelr.add(SpatialDropout1D(0.4))
modelr.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
modelr.add(Dense(2,activation='sigmoid'))
modelr.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(modelr.summary())

Y = pd.get_dummies(df['comp_score']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

batch_size = 32
modelr.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 2)

validation_size = 800

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = modelr.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
texter = "trump bad really dumb"

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(twitter_sample['correct_size'].values)
X = tokenizer.texts_to_sequences(twitter_sample['correct_size'].values)
X = pad_sequences(X)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(twitter_sample['correct_size'].values)
X = tokenizer.texts_to_sequences(twitter_sample['correct_size'].values)
X = pad_sequences(X)

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(twitter_sample['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 2)

In [ ]:
validation_size = 800

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

In [ ]:
twt = "@realDonaldTrump delivered the highest ratings in the history of presidential debates. #Trump2016"

def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

nlp = spacy.load("en_core_web_sm")

twt = twt.lower()
twt = give_emoji_free_text(twt)
my_list = ""
doc = nlp(twt)
    #print(doc)
   #print('1')
for token in doc:
    if token.text[0:2] == "rt":
        my_list == my_list
    else:
        lemm = token.lemma_
        poss = token.pos_
        stopp = token.is_stop
        thisdict = {
            "lem": lemm,
            "pos": poss,
            "stop": stopp
        }
        if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
            my_list += thisdict["lem"] + ' '
            
twt = my_list

#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

In [ ]:
df.info()

In [ ]:
reddit_sentiment['comments'] = list_of_reddit_comments

In [ ]:
print(len(list_of_reddit_comments))

In [ ]:
twitter_sample = twitter_sample.head(500)

In [ ]:
youtube_sample['correct_length']

list_of_twitter_comments = youtube_sample['correct_length'].values
print(len(list_of_twitter_comments))
list_of_sentiment = []

for comment in list_of_twitter_comments:
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt = comment
    twt = tokenizer.texts_to_sequences(twt)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
    #print(twt)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):
        list_of_sentiment.append("negative")
    elif (np.argmax(sentiment) == 1):
        list_of_sentiment.append("positive")
        
print(len(list_of_sentiment))
youtube_sentiment = pd.DataFrame(data=list_of_twitter_comments, columns = ['comments'])
youtube_sentiment['sentiment'] = list_of_sentiment
youtube_sentiment['comments'] = list_of_twitter_comments

In [ ]:
youtube_sentiment.to_csv('youtube_sentiment.csv', index=False)

In [ ]:
list_of_reddit_comments = df['correct_size'].values
print(len(list_of_reddit_comments))
list_of_sentiment = []

for comment in list_of_reddit_comments:
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt = comment
    twt = tokenizer.texts_to_sequences(twt)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
    #print(twt)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):
        list_of_sentiment.append("negative")
    elif (np.argmax(sentiment) == 1):
        list_of_sentiment.append("positive")
        
print(len(list_of_sentiment))
reddit_sentiment = pd.DataFrame(data=list_of_reddit_comments, columns = ['comments'])
reddit_sentiment['sentiment'] = list_of_sentiment
reddit_sentiment['comments'] = list_of_reddit_comments

In [ ]:
reddit_sentiment = pd.DataFrame(data=list_of_reddit_comments, columns = ['comments'])
reddit_sentiment['sentiment'] = list_of_sentiment
reddit_sentiment['comments'] = df['correct_size']

In [ ]:
reddit_sentiment.dropna(inplace=True)
reddit_sentiment.info()

In [ ]:
reddit_sentiment.to_csv('reddit_sentiment.csv', index=False)

In [ ]:
reddit_sentiment['comments'] = df['correct_size']
reddit_sentiment['sentiment'] = list_of_sentiment

In [ ]:
reddit_sentiment.info()

In [ ]:
testing_sample = pd.read_csv("/kaggle/input/twitter/Sentiment.csv")

testing_sample.drop(columns=['relevant_yn'], inplace=True)
testing_sample.drop(columns=['relevant_yn_confidence'], inplace=True)
testing_sample.drop(columns=['subject_matter'], inplace=True)
testing_sample.drop(columns=['subject_matter_confidence'], inplace=True)
testing_sample.drop(columns=['candidate_gold'], inplace=True)
testing_sample.drop(columns=['relevant_yn_gold'], inplace=True)
testing_sample.drop(columns=['retweet_count'], inplace=True)
testing_sample.drop(columns=['sentiment_gold'], inplace=True)
testing_sample.drop(columns=['subject_matter_gold'], inplace=True)
testing_sample.drop(columns=['tweet_coord'], inplace=True)
testing_sample.drop(columns=['tweet_created'], inplace=True)
testing_sample.drop(columns=['tweet_id'], inplace=True)
testing_sample.drop(columns=['user_timezone'], inplace=True)
testing_sample.drop(columns=['name'], inplace=True)
testing_sample.drop(columns=['tweet_location'], inplace=True)
testing_sample = testing_sample.drop(testing_sample[testing_sample.candidate != 'Donald Trump'].index)
testing_sample = testing_sample[testing_sample.sentiment != "Neutral"]
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

testing_sample['text'] = testing_sample['text'].apply(lambda x:clean_text(x))

import emoji
import spacy
import stop_words
s = testing_sample['text'].values
list_of_list = []
#print(s)
def give_emoji_free_text(text):
    return emoji.get_emoji_regexp().sub(r'', text)

nlp = spacy.load("en_core_web_sm")
for string in s:
    string = string.lower()
    string = give_emoji_free_text(string)
    my_list = ""
    doc = nlp(string)
    #print(doc)
   #print('1')
    for token in doc:
        if token.text[0:2] == "rt":
            my_list == my_list
        else:
            lemm = token.lemma_
            poss = token.pos_
            stopp = token.is_stop
            thisdict = {
                "lem": lemm,
                "pos": poss,
                "stop": stopp
            }
            if (thisdict["stop"] == False) & ((thisdict["pos"] == 'PROPN') or (thisdict["pos"] == 'VERB') or (thisdict["pos"] == 'NOUN') or (thisdict["pos"] == 'NUM') or (thisdict["pos"] == 'ADV') or (thisdict["pos"] == 'ADJ')):
                my_list += thisdict["lem"] + ' '
            
    list_of_list.append(my_list)

#print(list_of_list.values)
testing_sample['processing_text'] = list_of_list

testing_sample.drop(columns=['id'], inplace=True)
testing_sample.drop(columns=['candidate'], inplace=True)
testing_sample.drop(columns=['candidate_confidence'], inplace=True)
testing_sample.drop(columns=['sentiment_confidence'], inplace=True)
testing_sample.drop(columns=['text'], inplace=True)
testing_sample.head(10)
testing_sample.info

In [ ]:
testing_sample = testing_sample.head(500)
testing_sample.info()

In [ ]:
test_result = []
gold_result = testing_sample['sentiment']
testing_set = np.asarray(testing_sample['processing_text'])


for opinion in testing_set:
    opinion = tokenizer.texts_to_sequences(opinion)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    opinion = pad_sequences(opinion, maxlen=28, dtype='int32', value=0)
    #print(twt)
    sentiment = model.predict(opinion,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):    
         test_result.append("negative")
    elif (np.argmax(sentiment) == 1):       
        test_result.append("positive")
    
    
    #print(opinion)

In [ ]:
print(len(gold_result))

In [ ]:
for i in range(len(testing_set)):
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt = tokenizer.texts_to_sequences(twt)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
    #print(twt)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):
        print("negative")
    elif (np.argmax(sentiment) == 1):
        print("positive")
    test_result.append(classifier.classify(testing_set[i][0]))
    

In [ ]:
pos_count = 0
neg_count = 0
twitter = testingdf['sentiment']
for sentiment in twitter:
    if sentiment == 'positive':
        pos_count = pos_count + 1
    elif sentiment == 'negative':
        neg_count = neg_count + 1
        
print(neg_count)
print(pos_count)

In [ ]:
testing_df = pd.DataFrame(data=predictions, columns = ['preditcions'])

In [ ]:
twitter = youtube_sample['sentiment']

In [ ]:
#testing_df['gold_pred'] = list_of_g_s
testing_df['generic_pred'] = predictions_g

In [ ]:
testingdf['sm_predictions_traditional'] = predictions
testingdf['generic_predictions_traditional'] = predictions_g

In [ ]:
testingdf.head(300)

In [ ]:
import re, string
listdonetwo = []
testing = testingdf['generic_predictions_traditional']
print(type(testing[1]))
string = str(testing[1])
print((string))
for s in testing:
    strs = str(s)
    s = re.sub(r'[^\w\s]','',strs)
    listdonetwo.append(s)

    
print(len(listdonetwo))

In [ ]:
testingdf['sentiment'] = testingdf['sentiment'].replace('Positive', 'positive')
testingdf['sentiment'] = testingdf['sentiment'].replace('Negative', 'negative')

In [ ]:
testingdf['generic_predictions_traditional'] = listdonetwo

In [ ]:
testingdf['sm_predictions_traditional'] = listdone

In [ ]:
testing_df['sm_predictions_traditional'] = testing_df['sm_predictions_traditional'].replace('[positive]', 'positive')
testing_df['sm_predictions_traditional'] = testing_df['sm_predictions_traditional'].replace('[negative]', 'negative')
testing_df['preditcions'] = testing_df['preditcions'].replace('Positive', 'negative')
testing_df['preditcions'] = testing_df['preditcions'].replace('Negative', 'negative')

testing_df['gold_pred'] = testing_df['gold_pred'].replace(1, 'positive')
testing_df['gold_pred'] = testing_df['gold_pred'].replace(0, 'negative')
testing_df['gold_pred'] = testing_df['gold_pred'].replace('Positive', 'negative')
testing_df['gold_pred'] = testing_df['gold_pred'].replace('Negative', 'negative')

testing_df['generic_pred'] = testing_df['generic_pred'].replace(1, 'positive')
testing_df['generic_pred'] = testing_df['generic_pred'].replace(0, 'negative')
testing_df['generic_pred'] = testing_df['generic_pred'].replace('Positive', 'negative')
testing_df['generic_pred'] = testing_df['generic_pred'].replace('Negative', 'negative')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix   

test_result = testingdf['generic_predictions_traditional'].values
gold_result = testingdf['sentiment'].values

#print('\nClasification report:\n', classification_report(gold_result, test_result))
print('\nConfussion matrix:\n',confusion_matrix(gold_result, test_result))    

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt  
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

clf = SVC(random_state=0)
clf.fit(X_train, y_train)
SVC(random_state=0)
plot_confusion_matrix(clf, gold_result, test_result)  
plt.show()  


In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm = np.array([[   167,   6],
                                              [    116,  211]]), 
                      normalize    = False,
                      target_names = ['Positive', 'Negative'],
                      title        = "Confusion Matrix, Deep learning generic")

In [ ]:
CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)

print("Naive Bayes Algo accuracy percent:"+str((nltk.classify.accuracy(classifier, testing_set))*100)+"\n")

labels = {'pos', 'neg'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))

Testing phase

In [ ]:
twitter_sample.info()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(generic_sample['comments'],generic_sample['sentiment'],test_size = 0.2 , random_state = 0)

tvg=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvg.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvg.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

#training the model
mnbg=MultinomialNB()

#fitting the nb for tfidf features
mnb_tfidf=mnbg.fit(tv_train_reviews,y_train)
print(mnb_tfidf)

#pred_list = []
prediction = "Trump sucks ass"
prediction = [prediction]
prediction=tvg.transform(prediction)
#prediction = str(prediction)
#pred_list.append(prediction)
#prediction = [prediction]
mnb_tfidf_predict = mnbg.predict(prediction)
print(mnb_tfidf_predict)

x_train,x_test,y_train,y_test = train_test_split(twitter_sample['processing_text'],twitter_sample['sentiment'],test_size = 0.2 , random_state = 0)

tvt=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvt.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvt.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

#training the model
mnbt=MultinomialNB()

#fitting the nb for tfidf features
mnb_tfidf=mnbt.fit(tv_train_reviews,y_train)
print(mnb_tfidf)

#pred_list = []
prediction = "Trump sucks ass"
prediction = [prediction]
prediction=tvt.transform(prediction)
#prediction = str(prediction)
#pred_list.append(prediction)
#prediction = [prediction]
mnb_tfidf_predict = mnbt.predict(prediction)
print(mnb_tfidf_predict)

x_train,x_test,y_train,y_test = train_test_split(reddit_sample['comments'],reddit_sample['sentiment'],test_size = 0.2 , random_state = 0)

tvr=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvr.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvr.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

#training the model
mnbr=MultinomialNB()

#fitting the nb for tfidf features
mnb_tfidf=mnbr.fit(tv_train_reviews,y_train)
print(mnb_tfidf)

#pred_list = []
prediction = "Trump sucks ass"
prediction = [prediction]
prediction=tvr.transform(prediction)
#prediction = str(prediction)
#pred_list.append(prediction)
#prediction = [prediction]
mnb_tfidf_predict = mnbr.predict(prediction)
print(mnb_tfidf_predict)

Building the prediction set

In [ ]:
twitter_sample.info()
reddit_sample.info()
youtube_sample.info()
generic_sample.info()

In [ ]:
df.head()

In [ ]:
#redditredo.head()
#youtube_comments.head()
#twitter_sample.head()

In [ ]:
redditredo.info()
youtube_comments.info()
twitter_sample.info()

In [ ]:
twitter_test = twitter_sample.head(166)
youtube_test = youtube_comments.head(167)
reddit_test = redditredo.head(167)
prediction_set = []

list_of_t_c = twitter_test['correct_size'].values
list_of_t_s = twitter_test['sentiment'].values
list_of_r_c = reddit_test['comments'].values
list_of_r_s = reddit_test['comp_score'].values
list_of_y_c = youtube_test['text'].values
list_of_y_s = youtube_test['comp_score'].values
list_of_g_c = []
list_of_g_s = []

for thing in list_of_t_c:
    list_of_g_c.append(thing)
for thing in list_of_r_c:
    list_of_g_c.append(thing)
for thing in list_of_y_c:
    list_of_g_c.append(thing)

for thing in list_of_t_s:
    list_of_g_s.append(thing)
for thing in list_of_r_s:
    list_of_g_s.append(thing)
for thing in list_of_y_s:
    list_of_g_s.append(thing)    

print(len(list_of_g_c))
print(len(list_of_g_s))

In [ ]:
testingdf = pd.DataFrame(data=list_of_g_c, columns = ['comments'])
#testingdf = testingdf[testingdf.comments != "NaN"]
testingdf['sentiment'] = list_of_g_s
testingdf.info()

In [ ]:
twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('Positive', 'positive')
twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('Negative', 'negative')

In [ ]:
testingdf.head()

In [ ]:
 twt = comment
        twt = tokenizer.texts_to_sequences(twt)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
        #print(twt)
        sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
        if(np.argmax(sentiment) == 0):
            predictions.append("negative")
        elif (np.argmax(sentiment) == 1):
            predictions.append("positive")
            
            max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(twitter_sample['correct_size'].values)
X = tokenizer.texts_to_sequences(twitter_sample['correct_size'].values)
X = pad_sequences(X)

In [ ]:
redditredo.info()

In [ ]:
list_of_g_c = testingdf['comments'].values
list_of_g_c = list_of_g_c.tolist()
max_fatures = 2000

predictions_dl = []
for comment in list_of_g_c:
    index = list_of_g_c.index(comment)
    if index >= 0 & index < 166:
        twt = comment
        tokenizer = Tokenizer(num_words=max_fatures, split=' ')
        tokenizer.fit_on_texts(twitter_sample['correct_size'].values)
        twt = tokenizer.texts_to_sequences(twt)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
        #print(twt)
        sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
        if(np.argmax(sentiment) == 0):
            predictions_dl.append("negative")
        elif (np.argmax(sentiment) == 1):
            predictions_dl.append("positive")
    elif index >= 166 & index < 333:
        twt = comment
        tokenizer = Tokenizer(num_words=max_fatures, split=' ')
        tokenizer.fit_on_texts(redditredo['comments'].values)
       
        twt = tokenizer.texts_to_sequences(twt)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
        #print(twt)
        sentiment = modelr.predict(twt,batch_size=1,verbose = 2)[0]
        if(np.argmax(sentiment) == 0):
            predictions_dl.append("negative")
        elif (np.argmax(sentiment) == 1):
            predictions_dl.append("positive")
    elif index > 332:
        twt = comment
        tokenizer = Tokenizer(num_words=max_fatures, split=' ')
        tokenizer.fit_on_texts(youtube_comments['correct_length'].values)
        twt = tokenizer.texts_to_sequences(twt)
        #padding the tweet to have exactly the same shape as `embedding_2` input
        twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
        #print(twt)
        sentiment = modely.predict(twt,batch_size=1,verbose = 2)[0]
        if(np.argmax(sentiment) == 0):
            predictions_dl.append("negative")
        elif (np.argmax(sentiment) == 1):
            predictions_dl.append("positive")
        
        
print(len(predictions))
    

In [ ]:
testingdf.info()


In [ ]:
list_of_g_c = testingdf['comments'].values
list_of_g_c = list_of_g_c.tolist()
#list_of_g_c = list_of_g_c.asarray()
predictions = []
for comment in list_of_g_c:
    index = list_of_g_c.index(comment)
    if index >= 0 & index < 166:
        prediction = comment
        prediction = [prediction]
        prediction=tvt.transform(prediction)
       
        svc_tfidf_predict = svct.predict(prediction)
        print(svc_tfidf_predict)
        predictions.append(svc_tfidf_predict)
    elif index >= 166 & index < 333:
        prediction = comment
        prediction = [prediction]
        prediction=tvr.transform(prediction)
       
        svc_tfidf_predict = svcr.predict(prediction)
        predictions.append(svc_tfidf_predict)
    elif index > 332:
        prediction = comment
        prediction = [prediction]
        prediction=tvy.transform(prediction)
       
        svc_tfidf_predict = svcy.predict(prediction)
        predictions.append(svc_tfidf_predict)
        
        
print(len(predictions))
    

In [ ]:
testingdf['sm_predictions_traditional'] = predictions

In [ ]:
testingdf['generic_predictions_traditional'] = predictions_g

In [ ]:
predictions_g = []
for comment in list_of_g_c:
    prediction = comment
    prediction = [prediction]
    prediction=tvg.transform(prediction)
       
    mnb_tfidf_predict = svc.predict(prediction)
    predictions_g.append(mnb_tfidf_predict)
    
    twt = comment
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(generic_sentimentredo['comments'].values)
    twt = tokenizer.texts_to_sequences(twt)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
    #print(twt)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
    if(np.argmax(sentiment) == 0):
            predictions_dl.append("negative")
    elif (np.argmax(sentiment) == 1):
            predictions_dl.append("positive")
    

In [ ]:
predictions_g = []
for comment in list_of_g_c:
    prediction = comment
    prediction = [prediction]
    prediction=tvg.transform(prediction)
       
    mnb_tfidf_predict = mnbg.predict(prediction)
    predictions_g.append(mnb_tfidf_predict)

In [ ]:
print(len(predictions_g))

some other models idk

In [ ]:
twitter_sample.info()


In [ ]:
reddit_sample['sentiment'] = reddit_sample['sentiment'].replace('positive', 1)
reddit_sample['sentiment'] = reddit_sample['sentiment'].replace('negative', 0)

twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('positive', 1)
twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('negative', 0)
twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('Positive', 1)
twitter_sample['sentiment'] = twitter_sample['sentiment'].replace('Negative', 0)

generic_sample['sentiment'] = generic_sample['sentiment'].replace('positive', 1)
generic_sample['sentiment'] = generic_sample['sentiment'].replace('negative', 0)
generic_sample['sentiment'] = generic_sample['sentiment'].replace('Positive', 1)
generic_sample['sentiment'] = generic_sample['sentiment'].replace('Negative', 0)
#generic_sample.head(5)
#twitter_sample.head(5)
#reddit_sample.head(5)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.layers import Dense,LSTM
from keras.models import Sequential

In [ ]:
#generic_sentimentredo
twitter_sample.info()

In [ ]:
coms = generic_sentimentredo['comments'].values
print(type(coms[5000]))

In [ ]:
coms = twitter_sample['correct_size'].values
print(type(coms[5000]))

In [ ]:
sents = generic_sentimentredo['sentiment'].values
print(type(sents[5000]))

In [ ]:
unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)
    
#print(len(unused))

In [ ]:
twitter_sample.info()

In [ ]:
coms = twitter_sample['correct_size'].values
sents = twitter_sample['sentiment'].values


unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)

x_train,x_test,y_train,y_test = train_test_split(unused, sents,test_size = 0.2 , random_state = 0)

tvt=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvt.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvt.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

svct=SVC()
svct.fit(tv_train_reviews,y_train)

svc_tfidf_predict=svct.predict(tv_test_reviews)
#Accuracy score for tfidf features
svc_tfidf_score=accuracy_score(y_test,svc_tfidf_predict)
print("twitter svc _tfidf_score :",svc_tfidf_score)

coms = youtube_comments['correct_length'].values
sents = youtube_comments['comp_score'].values


unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)

x_train,x_test,y_train,y_test = train_test_split(unused, sents,test_size = 0.2 , random_state = 0)

tvy=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvy.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvy.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

svcy=SVC()
svcy.fit(tv_train_reviews,y_train)

svc_tfidf_predict=svcy.predict(tv_test_reviews)
#Accuracy score for tfidf features
svc_tfidf_score=accuracy_score(y_test,svc_tfidf_predict)
print("youtube svc _tfidf_score :",svc_tfidf_score)



coms = redditredo['comments'].values
sents = redditredo['comp_score'].values


unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)

x_train,x_test,y_train,y_test = train_test_split(unused, sents,test_size = 0.2 , random_state = 0)

tvr=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvr.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvr.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

svcr=SVC()
svcr.fit(tv_train_reviews,y_train)

svc_tfidf_predict=svcr.predict(tv_test_reviews)
#Accuracy score for tfidf features
svc_tfidf_score=accuracy_score(y_test,svc_tfidf_predict)
print("reddit svc _tfidf_score :",svc_tfidf_score)

coms = generic_sentimentredo['comments'].values
sents = generic_sentimentredo['sentiment'].values

#coms = youtube_comments['correct_length'].values
#sents = youtube_comments['comp_score'].values
unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)

x_train,x_test,y_train,y_test = train_test_split(unused, sents,test_size = 0.2 , random_state = 0)

tvg=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tvg.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tvg.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

svcg=SVC()
svcg.fit(tv_train_reviews,y_train)

svc_tfidf_predict=svcg.predict(tv_test_reviews)
#Accuracy score for tfidf features
svc_tfidf_score=accuracy_score(y_test,svc_tfidf_predict)
print("generic svc _tfidf_score :",svc_tfidf_score)

In [ ]:
#coms = generic_sentimentredo['comments'].values
#sents = generic_sentimentredo['sentiment'].values

coms = twitter_sample['correct_size'].values
sents = twitter_sample['sentiment'].values

#coms = df['correct_size'].values
#sents = df['comp_score'].values

#coms = youtube_comments['correct_length'].values
#sents = youtube_comments['comp_score'].values
unused = []
for stra in coms:
    
    strastring = str(stra)
    unused.append(strastring)
    

#coms = coms.asarray()
#sents = sents.asarray()

x_train,x_test,y_train,y_test = train_test_split(unused, sents,test_size = 0.2 , random_state = 0)

tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tv.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=0)

#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,y_train)
print(lr_tfidf)

lr_tfidf_predict=lr.predict(tv_test_reviews)
lr_tfidf_score=accuracy_score(y_test,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)


#training the model
mnb=MultinomialNB()
#fitting the nb for bag of words

#fitting the nb for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,y_train)
print(mnb_tfidf)

mnb_tfidf_predict=mnb.predict(tv_test_reviews)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

svc=SVC()
svc.fit(tv_train_reviews,y_train)

svc_tfidf_predict=svc.predict(tv_test_reviews)
#Accuracy score for tfidf features
svc_tfidf_score=accuracy_score(y_test,svc_tfidf_predict)
print("svc _tfidf_score :",svc_tfidf_score)


In [ ]:
twitter_sample.info()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(generic_sentimentredo['comments'],generic_sentimentredo['sentiment'],test_size = 0.2 , random_state = 0)

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(x_train)
#transformed test reviews
tv_test_reviews=tv.transform(x_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [ ]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=0)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,y_train)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,y_train)
print(lr_tfidf)

In [ ]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)

In [ ]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(y_test,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(y_test,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

In [ ]:
#Classification report for bag of words
lr_bow_report=classification_report(y_test,lr_bow_predict,target_names=['0','1'])
print(lr_bow_report)

#Classification report for tfidf features
lr_tfidf_report=classification_report(y_test,lr_tfidf_predict,target_names=['0','1'])
print(lr_tfidf_report)

In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the nb for bag of words
mnb_bow=mnb.fit(cv_train_reviews,y_train)
print(mnb_bow)
#fitting the nb for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,y_train)
print(mnb_tfidf)

In [ ]:
#pred_list = []
prediction = "Trump sucks ass"
prediction = [prediction]
prediction=tv.transform(prediction)
#prediction = str(prediction)
#pred_list.append(prediction)
#prediction = [prediction]
mnb_tfidf_predict = mnb.predict(prediction)
print(mnb_tfidf_predict)

In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)

In [ ]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(y_test,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(y_test,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

In [ ]:
mnb_bow_report = classification_report(y_test,mnb_bow_predict,target_names = ['0','1'])
print(mnb_bow_report)
mnb_tfidf_report = classification_report(y_test,mnb_tfidf_predict,target_names = ['0','1'])
print(mnb_tfidf_report)

tensorflow model #2

In [ ]:
model = Sequential()
model.add(Dense(units = 75 , activation = 'relu' , input_dim = tv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu')) 
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

model.summary()



In [ ]:

#tv_train_reviews = tv_train_reviews
model.fit(tv_train_reviews,y_train , epochs = 25)

In [ ]:
tv_test_reviews = tv_test_reviews.toarray()
model.evaluate(tv_test_reviews,y_test)[1]